In [20]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [22]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

# Comparacion de Resultados GRD y DEIS

En este cuadernillo se quieren calcular las diferencias entre las bases de egresos hospitalarios GRD y DEIS. Se compararán:

1. Cantidad de egresos hospitalarios
2. Ranking para cada uno de los diagnósticos

Este análisis se realizará a nivel Nacional y a nivel del Hospital del Tórax

In [37]:
# Lee ranking de egresos DEIS
ranking_egresos_nacionales_deis = pl.read_csv(
    "../data/external/ranking_nacional_egresos.csv", encoding="latin-1", separator=";"
)

# Lee ranking de egresos GRD
ranking_egresos_nacionales_grd = pl.read_csv(
    "../data/interim/ranking_nacionales_egresos_grd.csv", separator=";"
)

In [60]:
# Define columnas a analizar entre ambas bases
COLUMNAS_A_SELECCIONAR = [
    "ANIO_EGRESO",
    "COD_HOSPITAL",
    "DIAGNOSTICO1",
    "Capítulo",
    "Sección",
    "Categoría",
    "Descripción",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "n_egresos",
    "n_egresos_DEIS",
    "ranking_egresos",
    "ranking_grd_n_egresos",
    "dias_estada_totales",
    "dias_estada_totales_DEIS",
    "ranking_nacionales_n_egresos",
    "ranking_publicos_n_egresos",
]

# Une ambas bases de datos segun el anio, hospital y diagnostico principal
comparacion_grd_deis_pais = (
    ranking_egresos_nacionales_grd.join(
        ranking_egresos_nacionales_deis,
        left_on=["ANIO_EGRESO", "COD_HOSPITAL", "DIAGNOSTICO1"],
        right_on=["ANO_EGRESO", "ESTABLECIMIENTO_SALUD", "DIAG1"],
        how="inner",
        suffix="_DEIS",
    )
    .select(pl.col(COLUMNAS_A_SELECCIONAR))
    .to_pandas()
)

# Obtiene comparacion de egresos solamente para el Torax
comparacion_grd_deis_torax = comparacion_grd_deis_pais.query("COD_HOSPITAL == 112103").copy()

In [61]:
DIFERENCIAS_A_CALCULAR = [
    ("n_egresos", "n_egresos_DEIS"),
    ("ranking_egresos", "ranking_grd_n_egresos"),
]

# Calcula diferencias entre egresos y rankings a nivel pais y Torax
comparacion_grd_deis_torax = build_features.calculate_and_add_difference(
    comparacion_grd_deis_torax, DIFERENCIAS_A_CALCULAR
)

comparacion_grd_deis_pais = build_features.calculate_and_add_difference(
    comparacion_grd_deis_pais, DIFERENCIAS_A_CALCULAR
)

In [62]:
# Obtiene el promedio de las diferencias de egresos
diferencia_promedio_egresos_torax = comparacion_grd_deis_torax[
    "n_egresos_n_egresos_DEIS_difference"
].mean()

# Obtiene el promedio de las diferencias de rankings
diferencia_promedio_ranking_torax = comparacion_grd_deis_torax[
    "ranking_egresos_ranking_grd_n_egresos_difference"
].mean()

print(f"> {diferencia_promedio_egresos_torax=:.2f} egresos")
print(f"> {diferencia_promedio_ranking_torax=:.2f} posiciones")

> diferencia_promedio_egresos_torax=3.08 egresos
> diferencia_promedio_ranking_torax=4.46 posiciones


In [63]:
# Obtiene el promedio de las diferencias de egresos
diferencia_promedio_egresos_pais = comparacion_grd_deis_pais[
    "n_egresos_n_egresos_DEIS_difference"
].mean()

# Obtiene el promedio de las diferencias de rankings
diferencia_promedio_ranking_pais = comparacion_grd_deis_pais[
    "ranking_egresos_ranking_grd_n_egresos_difference"
].mean()

print(f"> {diferencia_promedio_egresos_pais=:.2f} egresos")
print(f"> {diferencia_promedio_ranking_pais=:.2f} posiciones")

> diferencia_promedio_egresos_pais=6.95 egresos
> diferencia_promedio_ranking_pais=6.14 posiciones


In [64]:
comparacion_grd_deis_pais.sort_values("n_egresos_n_egresos_DEIS_difference")

,ANIO_EGRESO,COD_HOSPITAL,DIAGNOSTICO1,Capítulo,Sección,Categoría,Descripción,GLOSA_ESTABLECIMIENTO_SALUD,n_egresos,n_egresos_DEIS,ranking_egresos,ranking_grd_n_egresos,dias_estada_totales,dias_estada_totales_DEIS,ranking_nacionales_n_egresos,ranking_publicos_n_egresos,n_egresos_n_egresos_DEIS_difference,ranking_egresos_ranking_grd_n_egresos_difference
61115,2019,112100,T420,"Cap.19 LESIONES TRAUMÁTICAS, ENVENENAMIENTOS ...","T36-T50 ENVENENAMIENTO, EFECTO ADVERSO E INFR...","T42 ENVENENAMIENTO POR ANTIEPILÉPTICOS, HIPNÓT...",Derivados de la hidantoína,"Hospital Del Salvador (Santiago, Providencia)",1,1,9,5.0,3,3,6,5.0,0,4.0
61172,2019,113130,T402,"Cap.19 LESIONES TRAUMÁTICAS, ENVENENAMIENTOS ...","T36-T50 ENVENENAMIENTO, EFECTO ADVERSO E INFR...",T40 ENVENENAMIENTO POR NARCÓTICOS Y PSICODISLÉ...,Otros opiáceos,Hospital Dr. Exequiel González Cortés (Santiag...,1,1,8,3.0,1,1,6,3.0,0,5.0
61173,2019,126100,T402,"Cap.19 LESIONES TRAUMÁTICAS, ENVENENAMIENTOS ...","T36-T50 ENVENENAMIENTO, EFECTO ADVERSO E INFR...",T40 ENVENENAMIENTO POR NARCÓTICOS Y PSICODISLÉ...,Otros opiáceos,Hospital Clínico de Magallanes Dr. Lautaro Nav...,1,1,8,3.0,4,4,6,3.0,0,5.0
61174,2019,104100,T402,"Cap.19 LESIONES TRAUMÁTICAS, ENVENENAMIENTOS ...","T36-T50 ENVENENAMIENTO, EFECTO ADVERSO E INFR...",T40 ENVENENAMIENTO POR NARCÓTICOS Y PSICODISLÉ...,Otros opiáceos,Hospital San José del Carmen (Copiapó),1,1,8,3.0,4,4,6,3.0,0,5.0
61176,2019,102100,T398,"Cap.19 LESIONES TRAUMÁTICAS, ENVENENAMIENTOS ...","T36-T50 ENVENENAMIENTO, EFECTO ADVERSO E INFR...",T39 ENVENENAMIENTO POR ANALGÉSICOS NO NARCÓTIC...,Otros analgésicos no narcóticos y antipirético...,Hospital Dr. Ernesto Torres Galdames (Iquique),3,3,1,1.0,7,8,1,1.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73097,2019,111100,O800,"Cap.15 EMBARAZO, PARTO Y PUERPERIO (O00-O9A)",O80-O84 CONTACTO PARA EL PARTO,O80 PARTO ÚNICO ESPONTÁNEO,"Parto único espontáneo, presentación cefálica ...","Hospital Clínico San Borja-Arriarán (Santiago,...",615,3116,15,2.0,1565,9370,2,2.0,2501,13.0
14284,2020,109100,O800,"Cap.15 EMBARAZO, PARTO Y PUERPERIO (O00-O9A)",O80-O84 CONTACTO PARA EL PARTO,O80 PARTO ÚNICO ESPONTÁNEO,"Parto único espontáneo, presentación cefálica ...","Complejo Hospitalario San José (Santiago, Inde...",657,4218,7,1.0,1499,10330,1,1.0,3561,6.0
73096,2019,109100,O800,"Cap.15 EMBARAZO, PARTO Y PUERPERIO (O00-O9A)",O80-O84 CONTACTO PARA EL PARTO,O80 PARTO ÚNICO ESPONTÁNEO,"Parto único espontáneo, presentación cefálica ...","Complejo Hospitalario San José (Santiago, Inde...",811,5182,6,1.0,1813,12986,1,1.0,4371,5.0
57614,2019,114101,Z511,Cap.21 FACTORES QUE INFLUYEN EN EL ESTADO DE ...,Z40-Z54 CONTACTOS PARA OTRA ATENCIÓN ESPECÍFI...,Z51 OTRA ATENCIÓN MÉDICA,Sesión de quimioterapia por tumor,Complejo Hospitalario Dr. Sótero del Río (Sant...,6087,732,2,3.0,1874,2085,4,3.0,5355,1.0


In [59]:
ranking_egresos_nacionales_deis

ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,Capítulo,Sección,Categoría,Descripción,DIAG1,n_egresos,dias_estada_totales,n_int_q,n_muertos,ranking_nacionales_n_egresos,total_nacionales_n_egresos,%_nacionales_n_egresos,ranking_publicos_n_egresos,total_publicos_n_egresos,%_publicos_n_egresos,ranking_privados_n_egresos,total_privados_n_egresos,%_privados_n_egresos,ranking_grd_n_egresos,total_grd_n_egresos,%_grd_n_egresos,ranking_interno_n_egresos,total_interno_n_egresos,%_interno_n_egresos
i64,i64,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str
2020,104103,"""Hospital Provincial del Huasco…","""Cap.21 FACTORES QUE INFLUYEN …","""Z69-Z76 PERSONAS QUE ENTRAN E…","""Z76 PERSONAS EN CONTACTO CON L…","""Personas en contacto con los s…","""Z769""",6,32,2,0,1,15,0.4,1.0,14.0,0.428571,null,null,null,1.0,6.0,1.0,null,null,null
2020,125102,"""Hospital Dr. Leopoldo Ortega R…","""Cap.21 FACTORES QUE INFLUYEN …","""Z69-Z76 PERSONAS QUE ENTRAN E…","""Z76 PERSONAS EN CONTACTO CON L…","""Personas en contacto con los s…","""Z769""",3,8,0,0,2,15,0.2,2.0,14.0,0.214286,null,null,null,null,null,null,null,null,null
2020,125103,"""Hospital Lord Cochrane""","""Cap.21 FACTORES QUE INFLUYEN …","""Z69-Z76 PERSONAS QUE ENTRAN E…","""Z76 PERSONAS EN CONTACTO CON L…","""Personas en contacto con los s…","""Z769""",2,349,0,0,3,15,0.133333,3.0,14.0,0.142857,null,null,null,null,null,null,null,null,null
2020,116101,"""Hospital de Teno""","""Cap.21 FACTORES QUE INFLUYEN …","""Z69-Z76 PERSONAS QUE ENTRAN E…","""Z76 PERSONAS EN CONTACTO CON L…","""Personas en contacto con los s…","""Z769""",1,1,0,0,4,15,0.066667,4.0,14.0,0.071429,null,null,null,null,null,null,null,null,null
2020,107107,"""Hospital de Petorca""","""Cap.21 FACTORES QUE INFLUYEN …","""Z69-Z76 PERSONAS QUE ENTRAN E…","""Z76 PERSONAS EN CONTACTO CON L…","""Personas en contacto con los s…","""Z769""",1,140,0,0,4,15,0.066667,4.0,14.0,0.071429,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2001,111200,"""Hospital Clínico Universidad C…",null,null,null,null,null,1,2,1,0,3,10,0.1,null,null,null,1.0,1.0,1.0,null,null,null,null,null,null
2001,113180,"""Hospital El Pino (Santiago, Sa…",null,null,null,null,null,1,8,0,0,3,10,0.1,3.0,9.0,0.111111,null,null,null,2.0,7.0,0.142857,null,null,null
2001,103100,"""Hospital Dr. Leonardo Guzmán (…",null,null,null,null,null,1,4,0,0,3,10,0.1,3.0,9.0,0.111111,null,null,null,2.0,7.0,0.142857,null,null,null
